In [10]:
import numpy as np
from cookiecutter.main import cookiecutter
import collections
from pathlib import Path
import json
import shutil

In [16]:
SEED = 178 
WORLD_NAME = 'tomato_gen'
MODEL_NAME_PREFIX = 'tomato'
OUT_PATH = Path(Path.cwd() / '../generated/test06').resolve()
MODEL_TEMPLATE = Path(Path.cwd() / '../templates/tomato_model').resolve()
WORLD_TEMPLATE = Path(Path.cwd() / '../templates/tomato_world').resolve()
ROW_COUNT = 8
ROW_LENGTH = 33
ROW_DIST = 2.5
CROP_DIST = 0.3

shutil.rmtree(OUT_PATH, ignore_errors=True)
np.random.seed(SEED)


In [17]:
# helper class to build the markers.json
class Markers:
    markers = []
    last_id = 0

    @staticmethod
    def next_id():
        Markers.last_id += 1
        return Markers.last_id

    @staticmethod
    def reset():
        Markers.markers = []
    
    @staticmethod
    def add_plant(x, y, z):
        id = Markers.next_id()
        Markers.markers.append({
            'marker_type': 'PLANT',
            'id': id,
            'translation': [x, y, z]
        })
        return id
    

    @staticmethod
    def add_fruit(x, y, z, plant_id):
        id = Markers.next_id()
        Markers.markers.append({
            'marker_type': 'FRUIT',
            'id': id,
            'translation': [x, y, z],
            'plant_id': plant_id
        })
        return id
    
    @staticmethod
    def dumps():
        return json.dumps(Markers.markers, indent=4)

In [18]:
ROW_COUNT *= 2
LL = ROW_LENGTH * CROP_DIST
print(ROW_COUNT, LL)

16 9.9


In [19]:
models = {'list': []}
Markers.reset()

for x in range(ROW_COUNT):
    for y in range(ROW_LENGTH):
        model_name = 'tomato_{}'.format(x * ROW_LENGTH + y)

        cookiecutter(str(MODEL_TEMPLATE),
             output_dir=str(OUT_PATH), 
             overwrite_if_exists=True, 
             no_input=True,
             extra_context={'world_name': WORLD_NAME, 'model_name': model_name})
        if x%2 ==0:
            x_pos, y_pos, z_pos = x * (ROW_DIST + CROP_DIST)*0.5, y * CROP_DIST, 0
        else:
            x_pos, y_pos, z_pos = (x-1) * (ROW_DIST + CROP_DIST)*0.5 + CROP_DIST, y * CROP_DIST, 0

        
        models['list'].append({
            'model': model_name,
            'name': model_name,
            'pose': '{} {} 0 0 0 0'.format(x_pos, y_pos)
        })
        x_pos += np.random.uniform(-0.1, 0.1)
        y_pos += np.random.uniform(-0.1, 0.1)
        seed = np.random.randint(10000)
        dir = (OUT_PATH / WORLD_NAME / model_name).resolve()
        dir_blender = (Path.cwd() / '../blender').resolve()
        blend = str(dir_blender / 'tomato_gen.blend')
        script = str(dir_blender / 'tomato_gen.py')
        ! blender $blend --background --python $script -- --model_dir $dir --seed $seed

        plant_id = Markers.add_plant(x_pos, y_pos, z_pos)
        with open(dir / 'markers.json') as markers_file:
            plant_markers = json.load(markers_file)
            for marker in plant_markers:
                if marker['marker_type'] == 'FRUIT':
                    Markers.add_fruit(
                        marker['translation'][0] + x_pos,
                        marker['translation'][1] + y_pos,
                        marker['translation'][2] + z_pos,
                        plant_id
                    )
            

cookiecutter(str(WORLD_TEMPLATE),
             output_dir=str(OUT_PATH), 
             overwrite_if_exists=True, 
             no_input=True,
             extra_context={'world_name': WORLD_NAME, 'models': models})

with open(OUT_PATH / WORLD_NAME / 'markers.json', 'w') as outfile:
    json.dump(Markers.markers, outfile, indent=4, sort_keys=True)

Blender 3.6.1 (hash 8bda729ef4dc built 2023-07-17 23:31:33)
Read blend: "/home/angel/world_gen/blender/tomato_gen.blend"
/home/angel/world_gen/blender/tomato_gen.py:146: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.duplicate"
  bpy.ops.object.duplicate({
/home/angel/world_gen/blender/tomato_gen.py:200: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.parent_clear"
  bpy.ops.object.parent_clear(
/home/angel/world_gen/blender/tomato_gen.py:203: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.delete"
  bpy.ops.object.delete({'selected_objects': [ob]}, use_global=True)
/home/angel/world_gen/blender/tomato_gen.py:205: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.join"
  bpy.ops.object.join({
/home/angel/world_g


Blender quit
Blender 3.6.1 (hash 8bda729ef4dc built 2023-07-17 23:31:33)
Read blend: "/home/angel/world_gen/blender/tomato_gen.blend"
/home/angel/world_gen/blender/tomato_gen.py:146: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.duplicate"
  bpy.ops.object.duplicate({
/home/angel/world_gen/blender/tomato_gen.py:205: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.join"
  bpy.ops.object.join({
/home/angel/world_gen/blender/tomato_gen.py:213: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "mesh.separate"
  bpy.ops.mesh.separate({
Collada export to: /home/angel/world_gen/generated/test06/tomato_gen/tomato_7/meshes/tomato.dae
Info: Exported 6 Objects

Blender quit
Blender 3.6.1 (hash 8bda729ef4dc built 2023-07-17 23:31:33)
Read blend: "/home/angel/world_gen/blender/tomato_gen.blend"
/hom


Blender quit
Blender 3.6.1 (hash 8bda729ef4dc built 2023-07-17 23:31:33)
Read blend: "/home/angel/world_gen/blender/tomato_gen.blend"
/home/angel/world_gen/blender/tomato_gen.py:146: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.duplicate"
  bpy.ops.object.duplicate({
/home/angel/world_gen/blender/tomato_gen.py:200: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.parent_clear"
  bpy.ops.object.parent_clear(
/home/angel/world_gen/blender/tomato_gen.py:203: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.delete"
  bpy.ops.object.delete({'selected_objects': [ob]}, use_global=True)
/home/angel/world_gen/blender/tomato_gen.py:205: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.join"
  bpy.ops.object.join({
/home

/home/angel/world_gen/blender/tomato_gen.py:200: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.parent_clear"
  bpy.ops.object.parent_clear(
/home/angel/world_gen/blender/tomato_gen.py:203: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.delete"
  bpy.ops.object.delete({'selected_objects': [ob]}, use_global=True)
/home/angel/world_gen/blender/tomato_gen.py:205: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.join"
  bpy.ops.object.join({
/home/angel/world_gen/blender/tomato_gen.py:213: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "mesh.separate"
  bpy.ops.mesh.separate({
Collada export to: /home/angel/world_gen/generated/test06/tomato_gen/tomato_21/meshes/tomato.dae
Info: Exported 7 Objects

Blender quit
Blender 3.

Collada export to: /home/angel/world_gen/generated/test06/tomato_gen/tomato_27/meshes/tomato.dae
Info: Exported 6 Objects

Blender quit
Blender 3.6.1 (hash 8bda729ef4dc built 2023-07-17 23:31:33)
Read blend: "/home/angel/world_gen/blender/tomato_gen.blend"
/home/angel/world_gen/blender/tomato_gen.py:146: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.duplicate"
  bpy.ops.object.duplicate({
/home/angel/world_gen/blender/tomato_gen.py:205: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.join"
  bpy.ops.object.join({
/home/angel/world_gen/blender/tomato_gen.py:213: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "mesh.separate"
  bpy.ops.mesh.separate({
Collada export to: /home/angel/world_gen/generated/test06/tomato_gen/tomato_28/meshes/tomato.dae
Info: Exported 6 Objects

Blender quit
Bl

...

/home/angel/world_gen/blender/tomato_gen.py:205: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.join"
  bpy.ops.object.join({
/home/angel/world_gen/blender/tomato_gen.py:213: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "mesh.separate"
  bpy.ops.mesh.separate({
Collada export to: /home/angel/world_gen/generated/test06/tomato_gen/tomato_496/meshes/tomato.dae
Info: Exported 7 Objects

Blender quit
Blender 3.6.1 (hash 8bda729ef4dc built 2023-07-17 23:31:33)
Read blend: "/home/angel/world_gen/blender/tomato_gen.blend"
/home/angel/world_gen/blender/tomato_gen.py:146: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.duplicate"
  bpy.ops.object.duplicate({
/home/angel/world_gen/blender/tomato_gen.py:205: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_

/home/angel/world_gen/blender/tomato_gen.py:200: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.parent_clear"
  bpy.ops.object.parent_clear(
/home/angel/world_gen/blender/tomato_gen.py:203: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.delete"
  bpy.ops.object.delete({'selected_objects': [ob]}, use_global=True)
/home/angel/world_gen/blender/tomato_gen.py:205: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.join"
  bpy.ops.object.join({
/home/angel/world_gen/blender/tomato_gen.py:213: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "mesh.separate"
  bpy.ops.mesh.separate({
Collada export to: /home/angel/world_gen/generated/test06/tomato_gen/tomato_504/meshes/tomato.dae
Info: Exported 7 Objects

Blender quit
Blender 3

/home/angel/world_gen/blender/tomato_gen.py:205: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.join"
  bpy.ops.object.join({
/home/angel/world_gen/blender/tomato_gen.py:213: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "mesh.separate"
  bpy.ops.mesh.separate({
Collada export to: /home/angel/world_gen/generated/test06/tomato_gen/tomato_511/meshes/tomato.dae
Info: Exported 6 Objects

Blender quit
Blender 3.6.1 (hash 8bda729ef4dc built 2023-07-17 23:31:33)
Read blend: "/home/angel/world_gen/blender/tomato_gen.blend"
/home/angel/world_gen/blender/tomato_gen.py:146: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.duplicate"
  bpy.ops.object.duplicate({
/home/angel/world_gen/blender/tomato_gen.py:200: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_

Collada export to: /home/angel/world_gen/generated/test06/tomato_gen/tomato_518/meshes/tomato.dae
Info: Exported 6 Objects

Blender quit
Blender 3.6.1 (hash 8bda729ef4dc built 2023-07-17 23:31:33)
Read blend: "/home/angel/world_gen/blender/tomato_gen.blend"
/home/angel/world_gen/blender/tomato_gen.py:146: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.duplicate"
  bpy.ops.object.duplicate({
/home/angel/world_gen/blender/tomato_gen.py:205: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.join"
  bpy.ops.object.join({
/home/angel/world_gen/blender/tomato_gen.py:213: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "mesh.separate"
  bpy.ops.mesh.separate({
Collada export to: /home/angel/world_gen/generated/test06/tomato_gen/tomato_519/meshes/tomato.dae
Info: Exported 6 Objects

Blender quit


/home/angel/world_gen/blender/tomato_gen.py:200: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.parent_clear"
  bpy.ops.object.parent_clear(
/home/angel/world_gen/blender/tomato_gen.py:203: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.delete"
  bpy.ops.object.delete({'selected_objects': [ob]}, use_global=True)
/home/angel/world_gen/blender/tomato_gen.py:205: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "object.join"
  bpy.ops.object.join({
/home/angel/world_gen/blender/tomato_gen.py:213: DeprecationWarning: Passing in context overrides is deprecated in favor of Context.temp_override(..), calling "mesh.separate"
  bpy.ops.mesh.separate({
Collada export to: /home/angel/world_gen/generated/test06/tomato_gen/tomato_526/meshes/tomato.dae
Info: Exported 7 Objects

Blender quit
Blender 3